### Using Scispacy

In [1]:
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz (15.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
patient_notes = pd.read_csv("patient_notes.csv")
patient_notes["pn_history"][patient_notes["case_num"] == 0][:10]

0    17-year-old male, has come to the student heal...
1    17 yo male with recurrent palpitations for the...
2    Dillon Cleveland is a 17 y.o. male patient wit...
3    a 17 yo m c/o palpitation started 3 mos ago; \...
4    17yo male with no pmh here for evaluation of p...
5    17 yo m, presenting with palpitations/increase...
6    17 yo male presents with 3-4 month history of ...
7    17 yo male presents with a 2-3 month history o...
8    HPI: 17 YO complains of intermittent heart pal...
9    Mr. Cleveland is a 17 yo M who presents to cli...
Name: pn_history, dtype: object

In [4]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
whole_case_num_zero[:1000]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std\n17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating.

In [5]:
import prepare_jag
clean_note = prepare_jag.basic_clean2(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

964036

In [1]:
print("works1")
import scispacy
import spacy
print("works2")

#from scispacy.linking import EntityLinker

print("works3")
sci_nlp = spacy.load("en_core_sci_sm")
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
print("works4")

works1
works2
works3
works4


In [2]:
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
sci_nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

doc = sci_nlp(clean_note)
# Let's look at a random entity!
# doc comes from above
entity = doc.ents[1]
print(f"Name: {entity}")

# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])

ValueError: [E002] Can't find factory for 'scispacy_linker' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, parser, beam_parser, entity_linker, ner, beam_ner, entity_ruler, lemmatizer, tagger, morphologizer, senter, sentencizer, textcat, textcat_multilabel, en.lemmatizer